In [99]:
import sklearn.datasets as datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

X_all, y_all = datasets.fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
print(X_all.shape, y_all.shape)


(70000, 784) (70000,)


In [100]:
X_all = X_all.astype("d")
X_all /= 255.0 
y_all = np.array([int(x) for x in y_all])
cls_ids = [int(x) for x in np.unique(y_all)]

Xt, yt = X_all[:50000], y_all[:50000]
Xc, yc = X_all[50000:], y_all[50000:]

In [166]:
cls_dict = {}
for i in range(10):
    X_i = Xt[yt == i]
    y_i = np.ones(len(X_i))
    for j in range(10):
        if i == j:
            continue
        X_j = Xt[yt == j]
        y_j = np.zeros(len(X_j))
        X_ij = np.vstack((X_i, X_j))
        y_ij = np.concatenate((y_i, y_j))
        reg = LogisticRegression(solver="liblinear", penalty="l2", 
                                 C=1.0, max_iter=300)
        reg.fit(X_ij, y_ij)
        print(i, j, 
              accuracy_score(y_ij, reg.predict(X_ij)),
              accuracy_score(yt, reg.predict(Xt)),
              accuracy_score(yc, reg.predict(Xc)),
             )
        cls_dict[(i,j)] = reg

0 1 0.9999057492931197 2e-05 0.0001
0 2 0.996969696969697 0.0005 0.001
0 3 0.9984052626333101 0.00034 0.00055
0 4 0.999387192319477 0.03712 0.03675
0 5 0.9947022674295402 0.0005 0.00055
0 6 0.9970656683193362 0.00446 0.0044
0 7 0.9999010586722074 0.02416 0.02125
0 8 0.9971352568037651 0.0006 0.0006
0 9 0.9992943548387097 0.02504 0.02465
1 0 0.9999057492931197 0.21218 0.2084
1 2 0.995397332331392 0.2104 0.2068
1 3 0.9966601725577512 0.21156 0.20775
1 4 0.9988611559267344 0.16124 0.1591
1 5 0.9991162608012569 0.21184 0.20815
1 6 1.0 0.19556 0.19355
1 7 0.9984336128259468 0.2001 0.1953
1 8 0.9913498098859316 0.2114 0.2074
1 9 0.9986874179636227 0.19344 0.1897
2 0 0.996969696969697 0.2117 0.2075
2 1 0.995397332331392 0.0018 0.0017
2 3 0.9815274605223955 0.06764 0.0678
2 4 0.9934873308232421 0.14166 0.13745
2 5 0.9915558370276546 0.13768 0.13675
2 6 0.9937493698961589 0.13282 0.13375
2 7 0.9943803608399882 0.07558 0.07205
2 8 0.9841997961264016 0.06048 0.0589
2 9 0.9959823222177582 0.14924 

In [167]:
def multi_classify(cls_dict, X):
    ys = []
    for i in range(10):
        y_i = np.zeros(len(X))
        for j in range(10):
            if i == j:
                continue
            cls = cls_dict[(i,j)]
            y = cls.predict(X)
            y_i += y
        ys.append(y_i)
    ys = tuple(ys)
    Ys = np.c_[ys]
    ret = np.argmax(Ys, axis=1)
    return ret

In [168]:
y = multi_classify(cls_dict, Xt)
print(accuracy_score(y, yt))
y = multi_classify(cls_dict, Xc)
print(accuracy_score(y, yc))

0.96278
0.9436


In [169]:
def transform(cls_dict, X):
    ys = []
    for i in range(10):
        for j in range(10):
            if i == j:
                continue
            cls = cls_dict[(i,j)]
            y = cls.predict(X)
            ys.append(y)
    ys = tuple(ys)
    U = np.c_[ys]
    return U

In [170]:
Ut = transform(cls_dict, Xt)
Uc = transform(cls_dict, Xc)

In [171]:
cls = LogisticRegression(max_iter=500, penalty="l2", C=1.0)
cls.fit(Ut, yt)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [172]:
print(accuracy_score(yt, cls.predict(Ut)))
print(accuracy_score(yc, cls.predict(Uc)))

0.96436
0.94485
